In [17]:
from IPython.display import HTML
HTML('''
    <style> body {font-family: "Roboto Condensed Light", "Roboto Condensed";} h2 {padding: 10px 12px; background-color: #DDE6D5; position: static; color: #ffffff; font-size: 40px;} .text_cell_render p { font-size: 15px; } .text_cell_render h1 { font-size: 30px; } h1 {padding: 10px 12px; background-color: #E5C1CD; color: #ffffff; font-size: 40px;} .text_cell_render h3 { padding: 10px 12px; background-color: #E8F3FF; position: static; color: #ffffff; font-size: 20px;} h4:before{ 
    content: "@"; font-family:"Wingdings"; font-style:regular; margin-right: 4px;} .text_cell_render h4 {padding: 8px; font-family: "Roboto Condensed Light"; position: static; font-style: italic; background-color: #FFB800; color: #ffffff; font-size: 18px; text-align: center; border-radius: 5px;}input[type=submit] {background-color: #E64626; border: solid; border-color: #734036; color: white; padding: 8px 16px; text-decoration: none; margin: 4px 2px; cursor: pointer; border-radius: 20px;}</style>
''')

# Introduction

1. To understand how Life Expectancy has developed, we will look at each continent's life expectancy over time.
    - **TODO: Line graph of life expectancy over time by continent with AVERAGE LINE**
2. We can see that Life Expectancy was fairly flat for all continents until 1870 (corresponding with Louis Pasteurs development of Germ Theory) where Europe, Oceania and the Americas begin their upward trend. However, for Asia and Africa, it is not until at least 1913 and 1925 respectively until they begin to rise. It should be noted that these figures are estimated, as data for Asia and Africa is very sparse until 1950, so it is difficult to find the exact point of change. 
3. This report aims to gain a deeper understanding of how Africa in particular was left behind in rising Life Expectancies, and what factors contribute to the continued gap between Africa and other continents.

# Cleaning

## Original Dataset Cleaning

In [28]:
import pandas as pd
import country_converter as coco
import numpy as np

In [30]:
#load inital dataset
raw = pd.read_csv("life-expectancy.csv")

In [32]:
#define entity categories
continent = ['Americas', 'Africa', 'Asia', 'Europe', 'Latin America and the Caribbean', 'Northern America', 'Oceania']
income_groups = ['High-income countries', 'Upper-middle-income countries', 'Middle-income countries', 'Lower-middle-income countries', 'Low-income countries', 'No income group available']
development = ['More developed regions', 'Small Island Developing States (SIDS)', 'Less developed regions', 'Less developed regions, excluding China', 'Less developed regions, excluding least developed countries', 'Least developed countries', 'Land-locked Developing Countries (LLDC)']

In [34]:
#create new df for continents, income level, development status
continents = raw[raw['Entity'].isin(continent) == True]
income = raw[raw['Entity'].isin(income_groups) == True]
develop = raw[raw['Entity'].isin(development) == True]

In [38]:
#create new df for countries
countries = raw[raw['Entity'].isin(development) == False]
countries = countries[countries['Entity'].isin(income_groups) == False]
countries = countries[countries['Entity'].isin(continent) == False]

In [55]:
#add continent column for countries df
converter = coco.CountryConverter()
countries['Continent'] = converter.convert(names=countries['Code'], src="ISO3", to="continent")

nan not found in ISO3
nan not found in ISO3
nan not found in ISO3
nan not found in ISO3
nan not found in ISO3
nan not found in ISO3
nan not found in ISO3
nan not found in ISO3
nan not found in ISO3
nan not found in ISO3
nan not found in ISO3
nan not found in ISO3
nan not found in ISO3
nan not found in ISO3
nan not found in ISO3
nan not found in ISO3
nan not found in ISO3
nan not found in ISO3
nan not found in ISO3
nan not found in ISO3
nan not found in ISO3
nan not found in ISO3
nan not found in ISO3
nan not found in ISO3
nan not found in ISO3
nan not found in ISO3
nan not found in ISO3
nan not found in ISO3
nan not found in ISO3
nan not found in ISO3
nan not found in ISO3
nan not found in ISO3
nan not found in ISO3
nan not found in ISO3
nan not found in ISO3
nan not found in ISO3
nan not found in ISO3
nan not found in ISO3
nan not found in ISO3
nan not found in ISO3
nan not found in ISO3
nan not found in ISO3
nan not found in ISO3
nan not found in ISO3
nan not found in ISO3
nan not fo

In [58]:
#find countries without continent match
a = countries[countries['Code'].isna() == True]
a = a[['Entity', 'Code']]
grouped = a.groupby(by="Entity").sum()

In [60]:
#update continent column
countries['Continent'] = countries['Continent'].str.replace('not found', 'Europe')

## Additional Datasets

### Disease Mortality Rates (2010 - 2021)

In [54]:
# load dataset (https://ghdx.healthdata.org/record/ihme-data/gbd-2021-cause-specific-mortality-1990-2021)
disease = pd.read_excel("disease-mortality-rates.XLSX")

In [63]:
#create a subset with only mortality rate per 100,000 people
disease = disease[['location_type', 'location_name', 'cause_name', '2010 (ASMR)', '2019 (ASMR)', '2020 (ASMR)', '2021 (ASMR)']]

#convert columns to numeric
cols = ['2010 (ASMR)', '2019 (ASMR)', '2020 (ASMR)', '2021 (ASMR)']
for i in cols:
    disease[i] = disease.loc[:, i].str.replace(r"\(.*\)","", regex=True)
    disease[i] = pd.to_numeric(disease.loc[:, i])

In [108]:
#create column for change from 2010 to 2021
disease.loc[:, 'change'] = (disease.loc[:, '2021 (ASMR)'] - disease.loc[:, '2010 (ASMR)'])

In [106]:
#create global dataset
globe = disease[disease['location_type'] == 'Global']

#create global grouped dataset by cause
globe_group = globe[['cause_name', '2010 (ASMR)', '2019 (ASMR)', '2020 (ASMR)', '2021 (ASMR)', 'change']].groupby(by='cause_name').mean().reset_index()

In [69]:
#create africa dataset
a = ['North Africa and Middle East', 'Central Sub-Saharan Africa', 'Eastern Sub-Saharan Africa', 'Southern Sub-Saharan Africa', 'Western Sub-Saharan Africa']
africa = disease[(disease['location_type'] == 'Region') & (disease['location_name'].isin(a) == True)]

#create grouped africa dataset by cause
ac = africa.copy()
ac = ac[['cause_name', '2010 (ASMR)', '2019 (ASMR)', '2020 (ASMR)', '2021 (ASMR)', 'change']]
africa_group = ac.groupby(by='cause_name').mean().reset_index()

In [103]:
#create column of difference between african vs global average in 2021
africa_group['diff'] = (africa_group['2021 (ASMR)'] - globe_group['2021 (ASMR)'])
africa_group = africa_group.sort_values(by='diff', ascending=False)

### Life Expectancy Decomposition (1990 - 2021)

In [80]:
#load dataset (https://cloud.ihme.washington.edu/s/6w3TkFaQw63Djnd?)
decomp = pd.read_excel("life-expectancy-decomp.XLSX")

### Death in Armed Conflicts (1989-2022)

In [85]:
#load df (Uppsala Conflict Data Program (2023); Natural Earth (2022) – processed by Our World in Data)
conflicts = pd.read_csv("deaths-in-armed-conflicts-by-country.csv")

In [93]:
#add continent column to df
converter = coco.CountryConverter()
conflicts['Continent'] = converter.convert(names=conflicts['Code'], src="ISO3", to="continent")

OWID_ABK not found in ISO3
OWID_ABK not found in ISO3
OWID_ABK not found in ISO3
OWID_ABK not found in ISO3
OWID_ABK not found in ISO3
OWID_ABK not found in ISO3
OWID_ABK not found in ISO3
OWID_ABK not found in ISO3
OWID_ABK not found in ISO3
OWID_ABK not found in ISO3
OWID_ABK not found in ISO3
OWID_ABK not found in ISO3
OWID_ABK not found in ISO3
OWID_ABK not found in ISO3
OWID_ABK not found in ISO3
OWID_ABK not found in ISO3
OWID_ABK not found in ISO3
OWID_ABK not found in ISO3
OWID_ABK not found in ISO3
OWID_ABK not found in ISO3
OWID_ABK not found in ISO3
OWID_ABK not found in ISO3
OWID_ABK not found in ISO3
OWID_ABK not found in ISO3
OWID_ABK not found in ISO3
OWID_ABK not found in ISO3
OWID_ABK not found in ISO3
OWID_ABK not found in ISO3
OWID_ABK not found in ISO3
OWID_ABK not found in ISO3
OWID_ABK not found in ISO3
OWID_ABK not found in ISO3
OWID_ABK not found in ISO3
OWID_ABK not found in ISO3
OWID_CZS not found in ISO3
OWID_CZS not found in ISO3
OWID_CZS not found in ISO3
O

In [94]:
#check for missed countries
conflicts['Continent'].unique()

array(['not found', 'Asia', 'Europe', 'Africa', 'America', 'Oceania'],
      dtype=object)

In [97]:
#remove non-states/former states from df
conflicts = conflicts[conflicts['Continent'] != 'not found']

In [99]:
#create df for african conflicts
africa = conflicts[conflicts['Continent'] == 'Africa']

### Healthcare Expenditure
World Health Organization - Global Health Observatory (2024) – processed by Our World in Data

### GDP

# Historical Context or How was Africa Left Behind? 

1. Africa was largely under colonial rule until the 1950's. Colonial management of the countries provided very little care to the Indigenous people, so the benefits of modern medicine were not distributed to the colonies. 
    - **TODO: Find a source**
2. In the 1950's the process of decolonisation begins. For some countries, Life Expectancy slowly and steadily increases (e.g.
    - **TODO: Line graph of the life expectancy of African countries from 1950-1975**
    - **TODO: Calculate pct change for these years (Africa vs Worldwide and compare)
3. I

# Potential Topics

## Specific Disease
*Can we explain the reason behind Africa's lagging life expectancy by some specific diseases?*
- *If Africa is being left behind on life expectancy growth, are there any specific diseases that can be accountable? Are they preventable - if it's not affecting other continents, they probably are.*
- *Has the Gates Foundation done much? (lol)*
- *Why did an extra 55 years to see the improved life expectancy outcomes following the discoveries of Louis Pasteur? Is a similar thing happening now?*

## War
*Can we explain a subset of countries life expectancy according to wars or political events like famines, etc and can we explain why the life expectancy will bounce back after these events? Additionally, will these political events have a longer term effect on population (e.g. maybe a country's life expectancy plummets due to a genocide, then the average life expectancy does bounce back but the population of the country will decrease. So the life expectancy may be high, but it is because it only includes the survivors.*
- *Does war have a significant effect on life expectancy from birth? Are there any countries that don't follow this upward trend?*

## How are Regions Left Behind?
- *We know that life expectancy has been going up for a long time, and the trend is found nearly everywhere. But, why is that Africa is still lagging way behind all other continents?*
- *Well, we can see that they had a much later start than every other continent - why?* **Records weren't really kept until 1950 when decolonisation really started. Colonialists weren't particularly interested in setting up medical infrastructure for the Indigenous people, and so they didn't get to feel the benefits of rapidly improving modern medicine until they started creating their own medical infrastructure.**
- *So what are the current effects? What diseases / causes of death are the most prevalent in Africa and why are they able to thrive where they have been eradicated in the rest of the world?* **TODO: look at life expectancy decomp + analyse the diseases that Africa is above average in**
- *Then they had the 'lost decade' - why did they fail to make it?*
- *We can also see, the biggest increase in deaths over time in Africa aside from Covid-19 (which they suffered the worst out of all continents) from **terrorism and conflict*** - *(this is led by Ethiopia/Somalia and the ethnic groups within)*
- *So we might want to think about how terrorism and conflict impacts life expectancy historically*
- *We might also want to think about how it affects overall population growth, and how that matches up with life expectancy*

## Storytelling

1. There is a trend of increasing life expectancy over time, fueled most clearly by the development of germ theory in 1870.
2. However, Africa does not start increasing until much later on - why?
    * Largely colonised, and colonial forces did not create much medical infrastructure for the Indigenous people, so the benefits of modern medicine were not shared. In the 50's, when decolonisation begins, then they start to develop these critical infrastructures.
    * So why are they still not meeting the global standard? Let's look into different diseases which are impacting the continent the most, comparative to other countries as well.
3. But disease is not the whole story, looking on a country level we can see the impact of conflict and terrorism in the region - focus on Sudan & Palestine
    * Can we find a correlation between deaths due to armed conflict and life expectancy?
    * Highlight how excessive conflict is in Africa comparative to other regions

**Pre 2000s / 90s**
1. There is a trend of increasing life expectancy over time, fueled most clearly by the development of germ theory in 1870.
2. However, Africa does not start increasing until much later on - why?
    a. Largely colonised, and colonial forces did not create much medical infrastructure for the Indigenous people, so the benefits of modern medicine were not shared. In the 50's, when decolonisation begins, then they start to develop these critical infrastructures.

**2000s Onward**

3. So why are they still not meeting the global standard? Let's look into different diseases which are impacting the continent the most, comparative to other countries as well.
4. But disease is not the whole story, looking on a country level we can see the impact of conflict and terrorism in the region - focus on Sudan & Palestine
    * Can we find a correlation between deaths due to armed conflict and life expectancy?
    * Highlight how excessive conflict is in Africa comparative to other regions

## TODO

### Analysis
- Create line graph of life expectancy by continent over time, with annotations on 1870 to mark the start of the upward trend and on 1935 to show when Africa begins to rise.
- Find a source (best would be a dataset) explaining medical infrastructure in colonial times / when health outcomes began to improve
- Disease analysis: point to specific diseases that are holding Africa back and their development over time (HIV, COVID-19, etc)
- Year on year change in life expectancy with year on year combat deaths - looking for correlation (time series analysis research)
- Case study on South Sudan 
- Map visualisation of combat deaths by continent (or country?)

### Report
- Convert to RMD
- Data provenance